In [23]:
# in case I need to install anything else
! pip install pymongo

   ---------------------------------------- 0.0/468.2 kB ? eta -:--:--
   ------------------- -------------------- 225.3/468.2 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 468.2/468.2 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.4 kB ? eta -:--:--
   ---------------------------------------- 300.4/300.4 kB 9.4 MB/s eta 0:00:00


In [15]:
import spotipy
from dotenv import load_dotenv
import os
import pandas as pd



In [16]:
#Authentication - without user
load_dotenv()

spot_id = os.getenv("SPOTCLIENT")
spot_key = os.getenv("SPOTSECRET")
client_credentials_manager = spotipy.SpotifyClientCredentials(client_id=spot_id, client_secret=spot_key)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
# from api documentation, but we are doing a no-user version right now
'''
sp_oauth = SpotifyOAuth(client_id=SPOTCLIENT, client_secret=SPOTSECRET, redirect_uri=REDIRECT_URI, scope=SCOPE)  
  
access_token = sp_oauth.get_access_token()  
refresh_token = sp_oauth.get_refresh_token()  
'''

# Pulling in BS data

In [17]:
# import the beat saver csv, 
bs_data = pd.read_csv('beatsaver_songs.csv')

# verification column seems unnecessary right now
# bs_data['spotified'] = False


In [22]:
bs_data['artist_genres']

0                                         ['bass trap']
1                                    ['art pop', 'pop']
2                                          ['speedrun']
3               ['anime rock', 'denpa-kei', 'vocaloid']
4        ['alternative metal', 'pop punk', 'rap metal']
5                                                    []
6                                                    []
7                        ['japanese vgm', 'sound team']
8                                          ['hardwave']
9                                       ['rhythm game']
10                         ['k-pop', 'k-pop boy group']
11                                 ['otacore', 'pixel']
12                      ["australian children's music"]
13                                                   []
14                       ['cyberpunk', 'dark clubbing']
15                                                   []
16                                            ['djent']
17    ['hip hop', 'melodic rap', 'philly rap', '

In [18]:
def get_spot_data(artist, title):
    '''Transforms Beat Saber Artist and Title into a spotify Track ID'''
    spot_search = sp.search(f'{artist} {title}')
    return spot_search['tracks']['items'][0]['id']

def all_spot_track_info(sp_id):
    '''Uses spotify Track ID to get details on that track'''
    return sp.track(sp_id)

def get_spot_audio(sp_id):
    '''Uses spotify track ID to get audio features of that track'''
    return sp.audio_features(sp_id)

def get_artist_info(art_id):
    '''Uses spotify Artist ID (from Track) to get artist data (for genre)'''
    return sp.artist(art_id)


In [19]:
# getting ahead of myself making it a loop
#for i in range(len(bs_data)):

for i in range(len(bs_data)):
    title = bs_data['title'].iloc[i]
    artist = bs_data['artist'].iloc[i]
    try:
        spot_result = get_spot_data(artist, title)
        track_dict = all_spot_track_info(spot_result)
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'spotify_id'] = spot_result
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'sp_title'] = track_dict['name']
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'sp_artist'] = track_dict['artists'][0]['name']
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'sp_artist_id'] = track_dict['artists'][0]['id']
        artist_info = get_artist_info(track_dict['artists'][0]['id'])
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'artist_genres'] = str(artist_info['genres'])
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'album'] = track_dict['album']['name']
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'album_released'] = track_dict['album']['release_date']
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'album_type'] = track_dict['album']['type']
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'sp_duration'] = track_dict['duration_ms']
        track_audio = get_spot_audio(spot_result)
        audio_vars = ['danceability', 'energy', 'key','loudness','mode', 'speechiness','acousticness','instrumentalness', 'liveness','valence','tempo','time_signature']
        for x in audio_vars:
            bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], f'sp_{x}'] = track_audio[0][x]
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'sp_popularity'] = track_dict['popularity']
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'artist_match'] = (bs_data['artist'].iloc[i].lower().strip() == bs_data['sp_artist'].iloc[i].lower().strip()) or ((bs_data['artist'].iloc[i].lower().strip() in bs_data['sp_artist'].iloc[i].lower().strip())) or (bs_data['sp_artist'].iloc[i].lower().strip()) in (bs_data['artist'].iloc[i].lower().strip())
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'title_match'] = (bs_data['title'].iloc[i].lower().strip() == bs_data['sp_title'].iloc[i].lower().strip()) or ((bs_data['title'].iloc[i].lower().strip() in bs_data['sp_title'].iloc[i].lower().strip())) or (bs_data['sp_title'].iloc[i].lower().strip()) in (bs_data['title'].iloc[i].lower().strip())
    except:
        bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'spotify_id'] = '000'
    


C:\Users\Sid\AppData\Local\Temp\ipykernel_19260\3778182616.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '6oFP5HK8RzPoW2puPSR3bz' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'spotify_id'] = spot_result
C:\Users\Sid\AppData\Local\Temp\ipykernel_19260\3778182616.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Bomb' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bs_data.loc[bs_data.bs_map_id == bs_data.bs_map_id[i], 'sp_title'] = track_dict['name']
C:\Users\Sid\AppData\Local\Temp\ipykernel_19260\3778182616.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Bishu' has dtype incompatible with float64, please explicitly cas

# cleanup and export

In [204]:
# fruits["favorite_fruits"] = fruits["favorite_fruits"].apply(eval)

# genres had to be transformed into a string, changing it back into a list
try:
    bs_data['artist_genres'] = bs_data['artist_genres'].apply(eval)
except:
    pass

bs_data['duration_difference'] = abs(bs_data['sp_duration']/1000 - bs_data['duration_seconds'])
bs_data['match_conclusion'] = ((bs_data[['artist_match','title_match']].all(axis=1)) | 
                               ((bs_data['duration_difference'] < 20) & (bs_data[['artist_match', 'title_match']].any(axis=1)))).astype(bool)

In [207]:
# establishing data file:
# bs_data.to_csv(r'all_song_data.csv', index=False)

# addint to data file:
bs_data.to_csv(r'all_song_data.csv', mode='a', header=False, index=False)